In [1]:
import pandas as pd
import numpy as np

## Example: US States Data

## 例子：美国州数据

> Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found at http://github.com/jakevdp/data-USstates/:

合并及联表操作在你处理多个不同数据来源时会经常出现。下面我们使用美国州及其人口数据作为例子来进行更加直观的说明。这些数据文件可以在http://github.com/jakevdp/data-USstates/ 中找到：

In [2]:
# 如果你没有数据文件，可以使用下面的命令下载它们
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

> Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

下面我们来载入三个相关的数据文件，使用Pandas的`read_csv()`函数：

In [3]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display(pop.head())
display(areas.head())
display(abbrevs.head())

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


> Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

有了数据之后，假如我们需要计算一个相对非常直接的结果：根据美国各州2010年人口密度进行排名。很显然我们有相关的数据，但是我们需要合并数据集才能找到结果。

> We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

我们先进行一个多对一的合并，将州全名和人口数据合并在一个`DataFrame`中。我们希望合并基于`pop`数据集的`state/region`列以及`abbreviation`数据集的`abbrevs`列。使用`how='outer'`来保证合并过程中不会因为不匹配的标签而丢失任何数据。

In [4]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1) # 移除冗余的列
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


> Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

让我们检查结果中是否有不匹配的情况，通过在数据集中寻找空值来查看：

In [5]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

> Some of the ``population`` info is null; let's figure out which these are!

一些人口`population`数据是空的；再来看看是哪些。

In [6]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


> It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source.

发现所有空的人口数据都是2000年前波多黎各的；这可能因为数据来源本来就没有这些数据造成的。

> More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

更重要的是，我们发现一些新的州`state`的数据也是空的，这意味着`abbrevs`列中不存在这些州的简称。再看看是哪些州有这种情况：

In [7]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

> We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

从上面的结果很容易发现：人口数据集中包括波多黎各（PR）和全美国（USA）的数据，而州简称数据集中却没有这两者数据。通过填充相应的数据可以很快解决这个问题：

In [8]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

> No more nulls in the ``state`` column: we're all set!

`state`列没有空值了：我们准备好了。

> Now we can merge the result with the area data using a similar procedure.
Examining our results, we will want to join on the ``state`` column in both:

下面我们可以将上面的结果数据集和面积数据集进行合并。研究两个数据集发现，我们需要在`state`列上进行数据集合并操作：

In [9]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


> Again, let's check for nulls to see if there were any mismatches:

再一次，我们检查一次空值，来看是否存在不匹配的情况：

In [10]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

> There are nulls in the ``area`` column; we can take a look to see which regions were ignored here:

面积`area`列有空值；我们看看是哪里出现的：

In [11]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

> We see that our ``areas`` ``DataFrame`` does not contain the area of the United States as a whole.
We could insert the appropriate value (using the sum of all state areas, for instance), but in this case we'll just drop the null values because the population density of the entire United States is not relevant to our current discussion:

结果显示面积数据集不包括整个美国的面积。我们可以为这个空值插入正确的值（使用所有州的面积数据之和），但是这个例子中我们只需要简单地移除空值数据即可，因为全美国的人口密度数据与我们前面的问题无关：

In [12]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


> Now we have all the data we need. To answer the question of interest, let's first select the portion of the data corresponding with the year 2000, and the total population.
We'll use the ``query()`` function to do this quickly (this requires the ``numexpr`` package to be installed; see [High-Performance Pandas: ``eval()`` and ``query()``](03.12-Performance-Eval-and-Query.ipynb)):

现在我们需要数据都已经准备好了。要回答前面那个问题，首先要选择出2010年相应的部分数据集以及不分年龄的全体人口数。我们使用`query()`函数来快速完成这项任务：

In [13]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


> Now let's compute the population density and display it in order.
We'll start by re-indexing our data on the state, and then compute the result:

下面我们可以计算人口密度并排序输出了。我们现将数据集按照`state`进行重新索引，然后计算结果：

In [14]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [15]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

> The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.
We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

结果是美国州根据2010年人口密度的排名，包括华盛顿特区和波多黎各，数据是每平方英里的居住人数。结果显示人口密度最稠密的地区是华盛顿特区（表中的the District of Columbia）；在其他的州中，人口密度最大的是新泽西。

> We can also check the end of the list:

我们也可以查看结果的最后部分：

In [16]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

> We see that the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.

结果显示密度最小的州，阿拉斯加，平均每平方英里略大于1个居民。

> This type of messy data merging is a common task when trying to answer questions using real-world data sources.
I hope that this example has given you an idea of the ways you can combine tools we've covered in order to gain insight from your data!

当使用真实世界数据回答这种问题的时候，这种数据集的合并是很常见的任务。作者希望这个例子能为你展示了Pandas数据集合并的工具的使用，并能在你的数据集中应用这些方法。